From 이수안 컴퓨터: http://suanlab.com/youtube/ml.html

# XGBoost (Extreme Gradient Boosting)

* 트리 기반의 앙상블 기법
* 분류에 있어서 다른 알고리즘보다 좋은 예측 성능을 보여줌
* XGBoost는 GBM 기반이지만, GBM의 단점인 느린 수행 시간과 과적합 규제 부재 등의 문제를 해결
* 병렬 학습이 지원되도록 구현되어 병렬 CPU 환경에서 빠르게 학습 가능

## 파이썬 기반 XGBoost

## XGBClassifier

### 붓꽃 데이터

### 와인 데이터

### 유방암 데이터

## XGBRegressor

### 보스턴 데이터

### 당뇨병 데이터

# LightGBM

* 빠른 학습과 예측 시간
* 더 적은 메모리 사용
* 범주형 특징의 자동 변환과 최적 분할
* 작은 데이터 세트에 사용하는 것은 비추천. 
  -- 행수 제한은 없지만 10,000 이상의 데이터에 사용하는 것을 권유
* overfitting에 민감, 적은 데이터에 대해서 과적합하기 쉬움

## LGBMClassifier

### 붓꽃 데이터

### 와인 데이터

### 유방암 데이터

## LGBMRegressor

### 보스턴 데이터

### 당뇨병 데이터